In [ ]:
import pandas as pd

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import airtable_utils as a
import pandas_utils as p
sys.path.pop()

import dotenv
import os
dotenv.load_dotenv()

rerun_query = 0

at_base_id = os.environ["AIRTABLE_DEVREL_BASE_ID"]

at_table_name = 'OP Project Data'

excl_projects = [
    '',
    # COMMENT OUT TO KEEP ALL PROJECTS
    # 'Op',
    # 'Optimism Governor'
]

In [ ]:
query_id = 2709489 #https://dune.com/queries/2709489
name = "Project Usage Trends - T28D"

if rerun_query == 0:
    projects = pd.read_csv('csv_outputs/' + name + '.csv')
else:
    # Dune query: https://dune.com/queries/2457627
    projects = d.get_dune_data(query_id, name = name, num_hours_to_rerun = 6, performance = "large")

In [ ]:
projects = projects[~projects['Team'].isin(excl_projects)]

In [ ]:
#reformat col names
projects = p.csv_columns_to_formatted(projects)

datestr = p.get_datestring_from_datetime(projects['Date'].iloc[0])
datestr = datestr.replace('-', '_')

p.mkdir_if_not_exists('uploads')
# deployers.to_csv('uploads/deployer_data_' + datestr + '.csv', index=False)

In [ ]:
print(projects.columns)

In [ ]:
# Delete Existing Data
# print('deleting existing data')S
a.delete_all_records(at_base_id,at_table_name)
# Append New Data
print('appending new data')
# deployers = deployers[deployers['Team'] == 'Cozy Finance'] #Test specific team
a.update_database(at_base_id,at_table_name, projects)
print('done')